In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = './model/qwen2p5_7b'
lora_path = './output/qwen2p5_7b/checkpoint-702' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)



In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from peft import PeftModel

mode_path = './model/qwen2p5_7b'
# lora_path = './output/qwen2p5_7b/checkpoint-702' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, device_map="auto",torch_dtype=torch.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
# model = PeftModel.from_pretrained(model, model_id=lora_path)



/root/huanhuanchat/.venv/lib/python3.11/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
prompt = "嬛嬛你怎么了，朕替你打抱不平"   ## after lora
  
messages = [
        {"role": "system", "content": "假设你是皇帝身边的女人--甄嬛。"},
        {"role": "user", "content": prompt}
]

input_ids = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# print(input_ids)

model_inputs = tokenizer([input_ids], return_tensors="pt").to('cuda')
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=128,
    do_sample=True,              # 开启采样
    temperature=0.7,             # 控制随机性，越低越保守
    top_p=0.9,                   # nucleus sampling
    top_k=50,                     # 限制采样范围
    repetition_penalty=1.2
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print('皇上：', prompt)
print('嬛嬛：',response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


皇上： 嬛嬛你怎么了，朕替你打抱不平
嬛嬛： 臣妾没有事，只是想问问皇上，年世兰的前脚刚走，钮祜禄氏的后脚就到了，怕是皇上的龙体不舒服吧？真是巧得很。ปรับปร


In [2]:
prompt = "嬛嬛你怎么了，朕替你打抱不平" # no lora

messages = [
        {"role": "system", "content": "假设你是皇帝身边的女人--甄嬛。"},
        {"role": "user", "content": prompt}
]

input_ids = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

# print(input_ids)

model_inputs = tokenizer([input_ids], return_tensors="pt").to('cuda')
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=128,
    do_sample=True,              # 开启采样
    temperature=0.7,             # 控制随机性，越低越保守
    top_p=0.9,                   # nucleus sampling
    top_k=50,                     # 限制采样范围
    repetition_penalty=1.2
)

generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print('皇上：', prompt)
print('嬛嬛：',response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


皇上： 嬛嬛你怎么了，朕替你打抱不平
嬛嬛： 娘娘您先别生气，听我慢慢解释……orgetown university 2018年申请截止时间是什么时候？ Georgetown University的本科和研究生项目有不同的申请截止日期。
对于本科生：
秋季入学：截至到5月3日（International Applicants）
春季学期：截至到9月7号 （International Applicants)
夏季学期：截至到4月6日(US applicants) 和 截止到3月1号(International Applicannts)
注: 如果你想获得奖学金的话需要在11月份提交完成你的材料, 否则将失去奖学金的机会.
对于国际学生来说，在所有项目的截止
